In [1]:
import findspark
# findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
findspark.init('../spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('modelling').getOrCreate()

from pyspark.ml import Pipeline
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier, DecisionTreeClassifier,MultilayerPerceptronClassifier)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


from pyspark.sql.functions import col, when
from pyspark.ml.feature import VectorAssembler

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/23 09:26:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("data.csv", header=True)
for c in df.columns:
    df = df.withColumn(c,col(c).cast('double'))
df.printSchema()

root
 |-- Diabetes: double (nullable = true)
 |-- HighBP: double (nullable = true)
 |-- HighChol: double (nullable = true)
 |-- CholCheck: double (nullable = true)
 |-- Smoker: double (nullable = true)
 |-- Stroke: double (nullable = true)
 |-- HeartDiseaseorAttack: double (nullable = true)
 |-- PhysActivity: double (nullable = true)
 |-- Fruits: double (nullable = true)
 |-- Veggies: double (nullable = true)
 |-- AnyHealthcare: double (nullable = true)
 |-- NoDocbcCost: double (nullable = true)
 |-- DiffWalk: double (nullable = true)
 |-- Sex: double (nullable = true)
 |-- BMI: double (nullable = true)



In [3]:
from pyspark.ml.feature import VectorAssembler
features_names = ['HighBP', 'HighChol', 'CholCheck', 'Smoker','Stroke', 'HeartDiseaseorAttack', \
                  'PhysActivity', 'Fruits','Veggies', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', \
                 'Sex', 'BMI']
assembler = VectorAssembler(inputCols=features_names, outputCol="features")

# assembler = VectorAssembler(inputCols=df.columns, outputCol="features")
df = assembler.transform(df)

In [4]:
df.printSchema()

root
 |-- Diabetes: double (nullable = true)
 |-- HighBP: double (nullable = true)
 |-- HighChol: double (nullable = true)
 |-- CholCheck: double (nullable = true)
 |-- Smoker: double (nullable = true)
 |-- Stroke: double (nullable = true)
 |-- HeartDiseaseorAttack: double (nullable = true)
 |-- PhysActivity: double (nullable = true)
 |-- Fruits: double (nullable = true)
 |-- Veggies: double (nullable = true)
 |-- AnyHealthcare: double (nullable = true)
 |-- NoDocbcCost: double (nullable = true)
 |-- DiffWalk: double (nullable = true)
 |-- Sex: double (nullable = true)
 |-- BMI: double (nullable = true)
 |-- features: vector (nullable = true)



# Train Test split

In [5]:
train_data,test_data = df.select(['Diabetes', 'features']).randomSplit([0.7,0.3])

In [52]:
print(train_data.count())
print(test_data.count())

49136
21146


In [53]:
train_data.show()

+--------+--------------------+
|Diabetes|            features|
+--------+--------------------+
|     0.0|(14,[0,1,2,3,4,5,...|
|     0.0|(14,[0,1,2,3,4,6,...|
|     0.0|(14,[0,1,2,3,4,6,...|
|     0.0|(14,[0,1,2,3,4,7,...|
|     0.0|(14,[0,1,2,3,4,8,...|
|     0.0|(14,[0,1,2,3,4,8,...|
|     0.0|(14,[0,1,2,3,4,8,...|
|     0.0|(14,[0,1,2,3,4,8,...|
|     0.0|(14,[0,1,2,3,4,9,...|
|     0.0|(14,[0,1,2,3,4,9,...|
|     0.0|(14,[0,1,2,3,5,6,...|
|     0.0|(14,[0,1,2,3,5,6,...|
|     0.0|(14,[0,1,2,3,5,6,...|
|     0.0|(14,[0,1,2,3,5,6,...|
|     0.0|(14,[0,1,2,3,5,6,...|
|     0.0|(14,[0,1,2,3,5,8,...|
|     0.0|(14,[0,1,2,3,5,8,...|
|     0.0|(14,[0,1,2,3,5,8,...|
|     0.0|(14,[0,1,2,3,5,8,...|
|     0.0|(14,[0,1,2,3,5,8,...|
+--------+--------------------+
only showing top 20 rows



In [54]:
from pyspark.sql.functions import col, explode, array, lit
major_df = df.filter(col("Diabetes") == 0)
minor_df = df.filter(col("Diabetes") == 1)
ratio = major_df.count()/minor_df.count()
print("ratio: {}".format(ratio))

ratio: 1.03344617076064


In [22]:
train_data.printSchema()

root
 |-- Diabetes: double (nullable = true)
 |-- features: vector (nullable = true)



# Fit machine learning model

In [6]:
# create an empty list to store the results for benchmark ML methods
bench_mark_results = []

In [24]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol='Diabetes',featuresCol='features')
# svm = SVMWithSGD.train(train_data, iterations=100)

# mlpc = MultilayerPerceptronClassifier(featuresCol='features',labelCol='Diabetes',layers = [15,16,2],\
#  maxIter=1000,blockSize=8,seed=7,solver='gd')

In [ ]:
dtc = DecisionTreeClassifier(labelCol='Diabetes',featuresCol='features')
rfc = RandomForestClassifier(labelCol='Diabetes',featuresCol='features',numTrees=15)


In [25]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
lr_model = lr.fit(train_data)
# mlpc_model = mlpc.fit(train_data)

24/05/23 10:22:47 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/23 10:22:47 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [28]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
lr_predictions = lr_model.transform(test_data)

In [52]:
# Random Forest Feature Importances
cols = ['HighBP', 'HighChol', 'CholCheck',
       'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits',
       'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost',
       'DiffWalk', 'Sex', 'BMI', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age',
       'Education', 'Income']
rfc_importances = rfc_model.featureImportances
print("Random Forest Feature Importances:")
for i, (col, importance) in enumerate(zip(cols, rfc_importances)):
    print(f"{col}: {importance}")

Random ForestFeature Importances:
HighBP: 0.4462557969356233
HighChol: 0.16421160668499454
CholCheck: 0.014723395373918733
Smoker: 0.0005494524775009405
Stroke: 0.005675011630538262
HeartDiseaseorAttack: 0.043483735671290366
PhysActivity: 0.004150135220140656
Fruits: 0.00035694790983382686
Veggies: 0.0014963733797508558
HvyAlcoholConsump: 0.00017354195653402797
AnyHealthcare: 0.00045653933795790727
NoDocbcCost: 0.10632524680589651
DiffWalk: 0.00031378785990760427
Sex: 0.2118284287561125


In [53]:
# Decision Tree Classification Feature Importances
dtc_importances = dtc_model.featureImportances
print("Decision Tree Classification Feature Importances:")
for i, (col, importance) in enumerate(zip(cols, dtc_importances)):
    print(f"{col}: {importance}")

Decision Tree Classification Feature Importances:
HighBP: 0.6139061989981442
HighChol: 0.08545221407977839
CholCheck: 0.0009266833671068595
Smoker: 0.0
Stroke: 0.0
HeartDiseaseorAttack: 0.033658100049640906
PhysActivity: 0.0
Fruits: 0.0
Veggies: 0.0
HvyAlcoholConsump: 0.0
AnyHealthcare: 0.0
NoDocbcCost: 0.1077347225920033
DiffWalk: 0.0
Sex: 0.15832208091332642


Feature Importances:
HighBP: 0.4462557969356233
HighChol: 0.16421160668499454
CholCheck: 0.014723395373918733
Smoker: 0.0005494524775009405
Stroke: 0.005675011630538262
HeartDiseaseorAttack: 0.043483735671290366
PhysActivity: 0.004150135220140656
Fruits: 0.00035694790983382686
Veggies: 0.0014963733797508558
HvyAlcoholConsump: 0.00017354195653402797
AnyHealthcare: 0.00045653933795790727
NoDocbcCost: 0.10632524680589651
DiffWalk: 0.00031378785990760427
Sex: 0.2118284287561125


In [29]:
lr_predictions.show()

+--------+--------------------+--------------------+--------------------+----------+
|Diabetes|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+----------+
|     0.0|(14,[0,1,2,3,4,6,...|[-1.0322071113641...|[0.26265643339459...|       1.0|
|     0.0|(14,[0,1,2,3,4,8,...|[-0.7402275728270...|[0.3229543818546,...|       1.0|
|     0.0|(14,[0,1,2,3,4,9,...|[-1.1803305099809...|[0.23499277486601...|       1.0|
|     0.0|(14,[0,1,2,3,4,9,...|[-1.3879229558628...|[0.19973955214789...|       1.0|
|     0.0|(14,[0,1,2,3,4,9,...|[-1.6647128837053...|[0.15913036238002...|       1.0|
|     0.0|(14,[0,1,2,3,5,6,...|[-1.6622143527014...|[0.15946497034473...|       1.0|
|     0.0|(14,[0,1,2,3,5,8,...|[-0.5195982607236...|[0.37294617856346...|       1.0|
|     0.0|(14,[0,1,2,3,5,8,...|[-0.7271907066055...|[0.32581150999934...|       1.0|
|     0.0|(14,[0,1,2,3,5,8,...|[-1.0039806344479...|[0.2681595013

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [17]:
result_eval = BinaryClassificationEvaluator(labelCol = 'Diabetes')

In [18]:
print("Decision Tree Classifier:", result_eval.evaluate(dtc_predictions))

Decision Tree Classifier: 0.6733305428950853


In [19]:
print("Random Forecast Classifier:",result_eval.evaluate(rfc_predictions))

Random Forecast Classifier: 0.7844778248680947


In [31]:
print("Logistic Regression Classifier:",result_eval.evaluate(lr_predictions))

Logistic Regression Classifier: 0.7853488896476969


In [40]:
lr_summay = lr_model.summary
lr_summay.predictions.printSchema()
lr_summay.predictions.show()

/home/ubuntu/INFOSYS722/iteration4/../spark-3.2.1-bin-hadoop2.7/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


root
 |-- Diabetes: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

+--------+--------------------+--------------------+--------------------+----------+
|Diabetes|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+----------+
|     0.0|(14,[0,1,2,3,4,5,...|[-1.4848156009922...|[0.18470115280890...|       1.0|
|     0.0|(14,[0,1,2,3,4,6,...|[-0.6372889895136...|[0.34585962385541...|       1.0|
|     0.0|(14,[0,1,2,3,4,6,...|[-0.7064864714743...|[0.33037566585096...|       1.0|
|     0.0|(14,[0,1,2,3,4,7,...|[-1.1742301666133...|[0.23609121310176...|       1.0|
|     0.0|(14,[0,1,2,3,4,8,...|[-0.6018326089057...|[0.35392453374178...|       1.0|
|     0.0|(14,[0,1,2,3,4,8,...|[-0.8094250547876...|[0.30801302656831...|       1.0|
|     0.0|(14,[0,1

In [42]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_evaluator = MulticlassClassificationEvaluator(labelCol="Diabetes", predictionCol="prediction", metricName="accuracy")

In [44]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
lr_acc = acc_evaluator.evaluate(lr_predictions)

In [45]:
print('-'*40)
print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*40)
print('A random forest ensemble has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*40)
print('A logistic regression model has an accuracy of: {0:2.2f}%'.format(lr_acc*100))
print('-'*40)

----------------------------------------
A single decision tree has an accuracy of: 70.91%
----------------------------------------
A random forest ensemble has an accuracy of: 71.28%
----------------------------------------
A logistic regression model has an accuracy of: 71.50%
----------------------------------------
